# PyTerrier - Learning to Rank

Ce TP à comme objetif la construction des pipelines dans [PyTerrier] (https://github.com/terrier-org/pyterrier). Toutes les expériences sont menées en utilisant le [corpus CORD19](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7251955/) et la [collection de tests TREC Covid](https://ir.nist.gov/covidSubmit/). Finalement, vous serez amené à refaire le processus avec une collection de questions-réponses.

Ce TP est divisé en deux. En particulier, dans la permière partie, vous allez expérimenter pour : 
 - comprendre le modèle de données PyTerrier ;
 - comprendre la notion générale de PyTerrier et les opérateurs permettant de combiner plusieurs recherches ;
 - entreprendre des expériences sur la collection de tests TREC Covid.

Dans la deuxième partie, vous ferez des expériences pour :
 - construire et apprendre des pipelines pour le Learning to Rank ;
 - évaluer et analyser des pipelines pour le Learning to Rank.


# Première partie

# Installation


In [2]:
!pip install -q --upgrade fastrank lightgbm==3.1.1
!pip install -q python-terrier

In [3]:
import pyterrier as pt
if not pt.started():
  pt.init()
cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



# Indexation


In [4]:
!rm -rf ./terrier_cord19/*

In [5]:
import os

pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

cord19/trec-covid documents:   0%|          | 0/192509 [00:00<?, ?it/s]

<ipython-input-5-59cedea60614>:11: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  index_ref = indexer.index(cord19.get_corpus_iter(),


21:11:41.927 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
21:12:55.221 [ForkJoinPool-1-worker-3] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.indexDocuments(BasicIndexer.java:270)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Inde

## Operateurs

Vous aurez remarqué que BatchRetrieve possède une méthode `transform()` qui prend en entrée un dataframe, et retourne un autre dataframe, qui est en quelque sorte une *transformation* du dataframe précédent (par exemple, un résultat de la recherche). En fait, `BatchRetrieve` n'est qu'un des nombreux objets similaires dans PyTerrier, que nous appelons [transformers](https://pyterrier.readthedocs.io/en/latest/transformer.html) (représentés par la classe `TransformerBase`).
Par exemple, on peut sauvegarder explicitement dans la varible ```tfidf``` l'application du modèle tfidf


In [6]:
tfidf = pt.BatchRetrieve(index_ref, wmodel="TF_IDF")

La capacité intéressante de tous les transformateurs est qu'ils peuvent être combinés à l'aide d'opérateurs Python (c'est ce qu'on appelle la *surcharge d'opérateurs*).

Concrètement, imaginez que vous voulez enchaîner des transformateurs ensemble - par exemple, trier des documents d'abord par tf puis retrier les documents par tfidf. Nous pouvons le faire en utilisant l'opérateur `>>` - nous appelons cela *composition*.

Il existe un certain nombre d'opérateurs PyTerrier - il y a plus d'exemples dans la [documentation PyTerrier sur les opérateurs](https://pyterrier.readthedocs.io/en/latest/operators.html)




## Q1 - Construction de pipelines

Créez un *ranker* (une classe que permet de chercher sur l'indèxe) qui effectue les opérations suivantes :
 - obtenir les 10 documents les mieux notés par tf (`wmodel="Tf"`)
 - obtenir les 10 documents les mieux notés par tfidf (`wmodel="TF_IDF"`)
 - ne remet en ordre que les documents trouvés dans l'un ou l'autre des paramètres de recherche précédents en utilisant BM25.

en utilisant des opérateurs PyTerrier combinant différentes instances de BatchRetrieve.

## Q2 - \# documents
Combien de documents sont récupérés par ce pipeline complet pour la requête `"chemical"` ?
> Hint: Si vous obtenez la bonne solution, le document avec le docno `"8hykq71k"` devrait avoir un score proche de $12.646089$ pour la requête `"chemical"`.

Conseils :
 - choisissez soigneusement vos [opérateurs PyTerrier](https://pyterrier.readthedocs.io/en/latest/operators.html)
 - vous ne devriez pas avoir à effectuer d'opérations sur les dataframes.

In [7]:
# réponse Q1
class Ranker:
     
  def search(self,index):
      BM25 = pt.BatchRetrieve(index, wmodel= "BM25")
      TF_IDF =  pt.BatchRetrieve(index, wmodel= "TF_IDF")
      TF =  pt.BatchRetrieve(index, wmodel= "Tf")
      pipe = (TF_IDF | TF) >> BM25
      return pipe


In [8]:
# réponse Q2
ranker = Ranker()
pipe = ranker.search(index_ref)
print(pipe)

Compose(Union(BR(TF_IDF), BR(Tf)), BR(BM25))


In [9]:
pipe.search("chemical")

,qid,docid,docno,rank,score,query
0,1,37771,jn5qi1jb,0,12.426309,chemical
1,1,15671,8hykq71k,1,12.413269,chemical
2,1,134305,0smev8vt,2,12.292890,chemical
3,1,142104,77c9ohxj,3,12.226076,chemical
4,1,87642,ck6clsty,4,12.155804,chemical
...,...,...,...,...,...,...
1368,1,7551,itfnu5dz,1368,3.261031,chemical
1369,1,14973,ie7mc8ti,1369,3.058071,chemical
1370,1,31086,e170ymw3,1370,2.563479,chemical
1371,1,88200,t473tbm9,1371,2.531990,chemical


# Évaluation dans un pipeline
À difference du TP3 où nous avons fait nous même l'évaluation, dans ce TP nous allons utiliser le module d'évaluation intégré pour les pipelines en pyterrier. Donc, pour faire des expériences (évaluer un modèle) sur pyterrier, on peut utiliser la classe `Experiment`. Voici le code qui évalue la performance de tfidf pour la collection cord19.  

In [10]:
# Télécharger la collection

topics = cord19.get_topics(variant='description')
qrels = cord19.get_qrels()


In [11]:
len(topics), len(qrels)

(50, 69318)

In [12]:
# Voici le code pour évaluer avec Experiment
from pyterrier.measures import *
pt.Experiment(
  #le pipeline
  [tfidf],
  topics,
  qrels,
  eval_metrics=[MAP, nDCG, nDCG@10],
  # on utilise TFIDF pour les tests statistiques
  baseline=0,
  names=["TFIDF"]
)



,name,AP,nDCG,nDCG@10,AP +,AP -,AP p-value,nDCG +,nDCG -,nDCG p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
0,TFIDF,0.188578,0.400915,0.638799,None,None,None,None,None,None,None,None,None


# Deuxième Partie  – Learning to Rank

Dans cette partie du TP, vous ferez la construction, l'apprentissage,  l'évaluation et l'analyse des pipelines d'apprentissage du classement (Learning to Rank).


Tout d'abord, répartissons les annotations (rankings) en ensembles d'entraînement, de validation et de test. TREC Covid ne comporte que 50 annotations, ce qui n'est pas beaucoup pour l'appprentissage. Nous allons répartir 30 annotations pour l'entraînement, 5 pour la validation et 15 pour l'évaluation. Nous examinerons également les differences statistiques, même si elle est petite, pour 15 annotations.

Nous n'allons classer que les 10 premiers documents pour chaque requête - nous espérons que l'apprentissage du classement pourra nous aider à retrier les 10 premiers documents pour qu'ils soient plus efficaces.


In [13]:
RANK_CUTOFF = 10
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)


test_qrels = qrels # seulement les annotations des topics en réponse sont utilisés, donc pas de problème si on utilise tout
train_qrels = qrels
valid_qrels = qrels

## Ensemble de caractéristiques

Définissons notre ensemble de caractéristiques.  Nous allons avoir un total de 6 caractéristiques :

1. le score abstrait de TFIDF ;
2. le résumé contient-il "coronavirus covid", noté par TFIDF ;
3. le score TFIDF sur le titre (même si nous ne l'avons pas indexé plus tôt !);
4. l'article a-t-il été publié en 2020 ? Ici, on a l'hypothèse que les articles récents étaient plus utiles pour cette tâche ;
5. L'article a-t-il un DOI, c'est-à-dire s'agit-il d'une publication officielle ?
6. le score de correspondance des coordonnées pour la requête - c'est-à-dire combien de termes de la requête apparaissent dans le résumé.

Plusieurs de ces caractéristiques nécessitent des métadonnées supplémentaires `["title", "date", "doi"]`. Heureusement, le jeu de données TREC Covid nous permet d'obtenir plus de métadonnées après l'indexation. Nous utilisons `pt.text.get_text(cord19, ["title", "date", "doi"])` pour récupérer ces colonnes de métadonnées supplémentaires.



In [14]:
ltr_feats1 = (tfidf % RANK_CUTOFF) >> pt.text.get_text(cord19, ["title", "date", "doi"]) >> (
    pt.transformer.IdentityTransformer()
    ** # score of text for query 'coronavirus covid'
    (pt.apply.query(lambda row: 'coronavirus covid') >> tfidf)
    ** # score of title (not originally indexed)
    (pt.text.scorer(body_attr="title", takes='docs', wmodel='TF_IDF') ) 
    ** # date 2020
    (pt.apply.doc_score(lambda row: int("2020" in row["date"])))
    ** # has doi
    (pt.apply.doc_score(lambda row: int( row["doi"] is not None and len(row["doi"]) > 0) ))
    ** # abstract coordinate match
    pt.BatchRetrieve(index_ref, wmodel="CoordinateMatch")
)

# for reference, lets record the feature names here too
fnames=["TFIDF", 'coronavirus covid', 'title', "2020", "hasDoi", "CoordinateMatch"]

Voyons le résultat pour une requête particulière. Nous pouvons voir que nous avons maintenant des colonnes supplémentaires de métadonnées de document `["title", "date", "doi"]`, ainsi que les colonnes `"features"` très importantes. Cela fait que le dataframe a le type $R_f$. En effet, c'est cette colonne que nous utilisons pour l'apprentissage.

In [15]:
ltr_feats1.search("Movie")

21:13:17.240 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


,qid,docid,docno,rank,score,query,title,date,doi,features
0,1,23347,qiwq0pe5,0,12.970648,Movie,Sentiment Analysis on Movie Scripts and Review...,2020-05-06,10.1007/978-3-030-49161-1_36,"[12.97064764826613, 0.0, 0.7458647062217206, 1..."
1,1,23343,vmetwotq,1,12.923459,Movie,Improving Movie Recommendation Systems Filteri...,2020-05-04,10.1007/978-3-030-49190-1_17,"[12.923458775835545, 0.0, 1.0350775514913675, ..."
2,1,78848,mmq44kwx,2,12.207116,Movie,"Smoking in top-grossing movies--United States,...",2011,,"[12.207115891520253, 0.0, 0.8744620693633965, ..."
3,1,70132,eynhsuz8,3,11.372312,Movie,The Post: A token woman leader's transformation,2020,10.1002/hrdq.21391,"[11.372311997669033, 1.499168191418449, 0.0, 1..."
4,1,24731,o7ckdng4,4,10.805086,Movie,Movies Emotional Analysis Using Textual Contents,2020-05-26,10.1007/978-3-030-51310-8_19,"[10.80508574921903, 0.0, 0.9277829272514087, 1..."
5,1,118935,25khbzk0,5,9.594602,Movie,CinemaGazer: a System for Watching Video at Ve...,2011-10-04,,"[9.594602481582845, 0.0, 0.0, 0.0, 0.0, 1.0]"
6,1,16001,3lhpdpiv,6,9.058977,Movie,The Aliens in Us and the Aliens Out There: Sci...,2013-11-17,10.1007/978-1-4614-7175-2_2,"[9.058977465455175, 0.0, 0.9880285718781234, 0..."
7,1,18722,gt3xayqp,7,8.579994,Movie,The Unfairness of Popularity Bias in Music Rec...,2020-03-24,10.1007/978-3-030-45442-5_5,"[8.579993656638317, 0.0, 0.0, 1.0, 1.0, 1.0]"
8,1,86076,opbwnnai,8,8.242861,Movie,Preparing for an influenza pandemic: mental he...,2009,,"[8.242860905714242, 0.0, 0.0, 0.0, 0.0, 1.0]"
9,1,5238,5z3pbbfb,9,8.042828,Movie,Characteristics of airborne Staphylococcus aur...,2014-05-07,10.1007/s10453-014-9342-6,"[8.042828188746888, 0.0, 0.0, 0.0, 1.0, 1.0]"


Nous pouvons également examiner les valeurs brutes des caractéristiques (dans ce cas, pour le premier document classé). Notez que le BM25 dans la colonne "score" ci-dessus est également la première valeur dans le tableau des caractéristiques (proche de 12), parce que nous avons utilisé un transformateur d'identité.

In [16]:
ltr_feats1.search("Movie").iloc[0]["features"]

21:13:17.634 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


array([12.97064765,  0.        ,  0.74586471,  1.        ,  1.        ,
        1.        ])

## Analysis

Ici nous analisons les performances de chaque caractéristique indépendamment. Pour cela, nous composons le pipeline de caractéristiques (`ltr_feats1`) avec `pt.ltr.feature_to_score(i)` pour un certain nombre de caractéristiques $i$.

In [17]:
pt.Experiment(
    [ltr_feats1 >> pt.ltr.feature_to_score(i) for i in range(len(fnames))],
    test_topics,
    test_qrels,  
    names=fnames,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "num_rel_ret"])

21:13:19.429 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 150 received 154, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


21:13:21.585 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 150 received 154, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


21:13:23.634 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


...

21:13:28.775 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 150 received 154, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


21:13:30.917 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 150 received 154, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


,name,map,ndcg,ndcg_cut_10,num_rel_ret
0,TFIDF,0.010519,0.047832,0.589368,96.0
1,coronavirus covid,0.010816,0.047858,0.584780,96.0
2,title,0.012200,0.054099,0.647125,96.0
3,2020,0.010942,0.048751,0.591473,96.0
4,hasDoi,0.010437,0.047810,0.575902,96.0
5,CoordinateMatch,0.010204,0.046617,0.570048,96.0


Il est intéressant de noter que la caractéristique "coronavirus covid" a obtenu un NDCG@10 de 0,5847. Il s'agit donc d'une baseline forte pour cette tâche.



## Learning 

Dans cette partie du TP, nous appliquons trois techniques différentes d'apprentissage du classement :

 - l'ascension des coordonnées à partir de FastRank, une technique linéaire par liste
 - les forêts aléatoires de `scikit-learn`, une technique d'arbre de régression par liste
 - LambdaMART de LightGBM, une technique d'arbre de régression par liste.

Dans chaque cas, nous prenons notre pipeline de caractéristiques, `ltr_feats1`, et nous le composons (`>>`) avec le modèle appris. Nous utilisons `pt.ltr.apply_learned_model()` qui sait comment gérer les différents apprenants.

Le pipeline complet est ensuite ajusté (appris) en utilisant `.fit()`, en spécifiant les annotations d'entraînement et les qrels. Il est important de noter que les étapes précédentes du pipeline (récupération et calcul des caractéristiques) sont appliquées aux annotations d'entraînement afin d'obtenir les résultats, qui sont ensuite transmis à la technique d'apprentissage par classement. LightGBM a un arrêt précoce activé, qui utilise un ensemble de annotations de validation - de la même manière, les annotations de validation sont transformés en résultats de validation.

Enfin, `%time` est la "commande magique" qui affiche le temps d'apprentissage pour chaque technique. L'apprentissage pour chaque technique prend < 30 secondes.

In [18]:
import fastrank

train_request = fastrank.TrainRequest.coordinate_ascent()

params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

ca_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(train_request, form='fastrank')

%time ca_pipe.fit(train_topics, train_qrels)

21:13:34.111 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 300 received 306, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


CPU times: user 23.4 s, sys: 87.9 ms, total: 23.5 s
Wall time: 15.2 s


In [19]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=400, verbose=1, random_state=42, n_jobs=2)

rf_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(rf, form='regression')


%time rf_pipe.fit(train_topics, train_qrels)

21:13:49.291 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 300 received 306, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    1.1s


CPU times: user 6.61 s, sys: 165 ms, total: 6.78 s
Wall time: 6 s


[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    2.6s finished


In [20]:
import lightgbm as lgb

# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

lmart_x_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(lmart_l, form="ltr", fit_kwargs={'eval_at':[10]})

%time lmart_x_pipe.fit(train_topics, train_qrels, valid_topics, valid_qrels)

21:13:55.466 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 300 received 306, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


21:13:57.029 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 50 received 54, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


[LightGBM] [Warning] early_stopping_round is set=5, early_stopping_rounds=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000902 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 268
[LightGBM] [Info] Number of data points in the train set: 24870, number of used features: 6
[Li

/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.9/dist-packages/lightgbm/engine.py:156: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


## Evaluation

Comparons maintenant nos pipelines de classement sur nos 15 annotations de teste versus la référence de base BM25. Dans tous les cas, nous ne classons que 10 résultats par requête, donc le MAP sera carrément plus faible. 

Nous allons rapporter le temps de réponse moyen (`"mrt"`) ainsi que les mesures MAP, NDCG et NDCG@10.

In [21]:
pt.Experiment(
    [tfidf % RANK_CUTOFF, ca_pipe, rf_pipe, lmart_x_pipe],
    test_topics,
    test_qrels, 
    names=["TFIDF",  "TFIDF + CA(6f)", "TFIDF + RF(6f)", "TFIDF + LMart(6f)"],
    baseline=0,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "mrt"])

21:13:59.821 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 150 received 154, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


21:14:01.729 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 150 received 154, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    0.2s finished


21:14:04.068 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 150 received 154, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


,name,map,ndcg,ndcg_cut_10,mrt,map +,map -,map p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,TFIDF,0.010519,0.047832,0.589368,56.896014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TFIDF + CA(6f),0.011900,0.052403,0.632980,126.243080,9.0,1.0,0.045345,9.0,1.0,0.010426,9.0,1.0,0.005700
2,TFIDF + RF(6f),0.011907,0.051449,0.627952,138.320349,8.0,4.0,0.038993,9.0,3.0,0.090934,9.0,3.0,0.078531
3,TFIDF + LMart(6f),0.011828,0.052731,0.622488,148.387550,6.0,5.0,0.217733,5.0,6.0,0.161192,5.0,6.0,0.223187


Question - les trois modèles appris ont pu améliorer NDCG@10 sur TFIDF ? Analysez les résultats, en indicant s'il y a des amélorations statistiques.

## Tâche pratique - Concaténation

Notre modèle appris a un faible rappel car seulement 10 documents sont reclassés. Créons une petite fonction, `append_baseline()`, qui peut ajouter les résultats de la ligne de base BM25 à la sortie du modèle appris. Ceci est défini en utilisant les [opérateurs de transformation] (https://pyterrier.readthedocs.io/en/latest/operators.html) (`^` et `%`).

Comme exercice, appliquez `append_baseline()` à chacun des pipelines de modèles appris définis ci-dessus, et rapportez le MAP et le NDCG calculés sur les 1000 résultats classés. 

Lequel des modèles appris résulte en une amélioration significative de MAP et NDCG ?

In [22]:
def append_baseline(system, baseline, max_results = 1000):
    return (system ^ baseline) % max_results

In [23]:
#réponse
lmart_rf= append_baseline(lmart_x_pipe,rf_pipe)
lmart_ca = append_baseline(lmart_x_pipe,ca_pipe)
rf_ca = append_baseline(rf_pipe,ca_pipe)
pt.Experiment(
    [tfidf % RANK_CUTOFF, lmart_ca, lmart_rf, rf_ca],
    test_topics,
    test_qrels, 
    names=["TFIDF",  "TFIDF + CA(6f)", "TFIDF + RF(6f)", "TFIDF + LMart(6f)"],
    baseline=0,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "mrt"])

21:14:07.422 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 150 received 154, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


21:14:09.297 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 150 received 154, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


21:14:41.822 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


...

21:15:12.333 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 150 received 154, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    0.2s finished


21:15:14.445 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 150 received 154, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


,name,map,ndcg,ndcg_cut_10,mrt,map +,map -,map p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,TFIDF,0.010519,0.047832,0.589368,65.765079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TFIDF + CA(6f),0.010766,0.049266,0.572290,2270.829934,5.0,7.0,0.865305,5.0,7.0,0.771759,5.0,7.0,0.766615
2,TFIDF + RF(6f),0.010766,0.049266,0.572290,2056.122892,5.0,7.0,0.865305,5.0,7.0,0.771759,5.0,7.0,0.766615
3,TFIDF + LMart(6f),0.010864,0.047999,0.577885,1966.962515,8.0,5.0,0.768285,9.0,4.0,0.966552,9.0,4.0,0.834835


# Tâche pratique

Utilisez les modèles implémentés pour cord19 dans une tâche de question-réponse. Dans ce contexte, les requêtes sont de questions et les documents sont des documents qui pourraient contenir la réponse. Notez que vous devez refaire l'indexation ainsi que les autres étapes étudiées dans ce TP. Voici un exemple du dataset à utiliser :

```json
"question": "Why are big companies like Apple or Google not included in the Dow Jones Industrial Average (DJIA) index?",

"answers":{
  "290156": { 
    "text":" That is a pretty exclusive club and for the most part they are not interested in highly volatile companies like Apple and Google. Sure, IBM is part of the DJIA, but that is about as stalwart as you can get these days. The typical profile for a DJIA stock would be one that pays fairly predictable dividends, has been around since money was invented, and are not going anywhere unless the apocalypse really happens this year. In summary, DJIA is the boring reliable company index." ,
    "timestamp": "Sep 11 '12 at 0:53"}
 }

```
Vous pouvez le télécharger en utilisant les lines de code ci-dessous.

In [24]:
fiqa = {}
fiqa['train'] = pt.datasets.get_dataset('irds:beir/fiqa/train') 
fiqa['valid'] = pt.datasets.get_dataset('irds:beir/fiqa/dev')
fiqa['test'] = pt.datasets.get_dataset('irds:beir/fiqa/test')

test_topics = fiqa['test'].get_topics(variant='text')
test_qrels = fiqa['test'].get_qrels()

train_topics = fiqa['train'].get_topics(variant='text')
train_qrels = fiqa['train'].get_qrels()

valid_topics = fiqa['valid'].get_topics(variant='text')
valid_qrels = fiqa['valid'].get_qrels()

In [ ]:
train_request = fastrank.TrainRequest.coordinate_ascent()

params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

ca_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(train_request, form='fastrank')

%time ca_pipe.fit(train_topics, train_qrels)

21:19:50.236 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


/usr/local/lib/python3.9/dist-packages/pyterrier/ops.py:242: UserWarning: Got number of results different expected from <pyterrier.batchretrieve.TextScorer object at 0x7f9502a89f40>, expected 54959 received 55321, feature scores for any missing documents be 0, extraneous documents will be removed
  warn("Got number of results different expected from %s, expected %d received %d, feature scores for any "


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=400, verbose=1, random_state=42, n_jobs=2)

rf_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(rf, form='regression')


%time rf_pipe.fit(train_topics, train_qrels)

In [ ]:
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

lmart_x_pipe = ltr_feats1 >> pt.ltr.apply_learned_model(lmart_l, form="ltr", fit_kwargs={'eval_at':[10]})

%time lmart_x_pipe.fit(train_topics, train_qrels, valid_topics, valid_qrels)

In [ ]:
pt.Experiment(
    [tfidf % RANK_CUTOFF, ca_pipe, rf_pipe, lmart_x_pipe],
    test_topics,
    test_qrels, 
    names=["TFIDF",  "TFIDF + CA(6f)", "TFIDF + RF(6f)", "TFIDF + LMart(6f)"],
    baseline=0,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "mrt"])

In [ ]:
lmart_rf= append_baseline(lmart_x_pipe,rf_pipe)
lmart_ca = append_baseline(lmart_x_pipe,ca_pipe)
rf_ca = append_baseline(rf_pipe,ca_pipe)
pt.Experiment(
    [tfidf % RANK_CUTOFF, lmart_ca, lmart_rf, rf_ca],
    test_topics,
    test_qrels, 
    names=["TFIDF",  "TFIDF + CA(6f)", "TFIDF + RF(6f)", "TFIDF + LMart(6f)"],
    baseline=0,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "mrt"])